In [0]:
! pip install twitter
! pip install emoji
! pip install nb_utils
! pip install gensim

     |████████████████████████████████| 61kB 2.4MB/s 
     |████████████████████████████████| 51kB 3.9MB/s 
  ERROR: Could not find a version that satisfies the requirement nb_utils (from versions: none)
ERROR: No matching distribution found for nb_utils


In [0]:
import random
import twitter
import emoji
import itertools
import pandas as pd
from itertools import chain
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
import keras.callbacks
import json

import os
#import nb_utils
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, Embedding, GlobalMaxPooling1D
from keras.models import Model
from keras.layers import Concatenate, Average

from gensim.models import Word2Vec

Using TensorFlow backend.


In [0]:
# Fill these in!

CONSUMER_KEY = 'xbMuxcJpRTiVGt2C2EYnA'
CONSUMER_SECRET = '2DbQTsvIptkPTdaUcos8DDvQH9fzO0hNjJpUT2uVzQ'
ACCESS_TOKEN = '7319442-EDm4CPxL7W4KkZcGWRMJNVHp88W5OH9vgblu898fg'
ACCESS_SECRET = '5ZxJSbqXhG7uhgXzTFWf9XhkfsxxinlPRXyDTzbA9w'

In [0]:
auth=twitter.OAuth(
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    token=ACCESS_TOKEN,
    token_secret=ACCESS_SECRET,
)

status_stream = twitter.TwitterStream(auth=auth).statuses

[x['text'] for x in itertools.islice(status_stream.sample(), 0, 5) if x.get('text')]

["RT @bayramsenocak: Cumhurbaşkanlığı Güvenlik ve Dış Politikalar Kurulu Üyesi Prof. Dr. Beril Dedeoğlu hocamıza Allah'tan rahmet, ailesine v…"]

In [0]:
status_stream = twitter.TwitterStream(auth=auth).statuses

def english_has_emoji(tweet):
    if tweet.get('lang') != 'en':
        return False
    return any(ch for ch in tweet.get('text', '') if ch in emoji.UNICODE_EMOJI)

%time tweets = list(itertools.islice(filter(english_has_emoji, status_stream.sample()), 0, 100))

CPU times: user 1.42 s, sys: 189 ms, total: 1.61 s
Wall time: 1min 4s


In [0]:
stripped = []
for tweet in tweets:
    text = tweet['text']
    emojis = {ch for ch in text if ch in emoji.UNICODE_EMOJI}
    if len(emojis) == 1:
        emoiji = emojis.pop()
        text = ''.join(ch for ch in text if ch != emoiji)
        stripped.append((text, emoiji))
len(stripped)

69

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:

all_tweets = pd.read_csv('/content/gdrive/My Drive/emojis.csv')
all_tweets['emoji'].value_counts()

😂    124823
❤     43218
😍     40566
😭     35714
😊     20076
🙄     17963
😩     16232
🔥     15453
🤔     15419
💕     12026
💯     11783
😘     11065
💀      9928
✨      9886
🙃      9405
👀      7842
😒      7019
☺      6871
😢      6846
😳      6716
💙      6616
😎      6349
😉      6272
😅      6133
😁      6010
😌      5759
😏      5623
💖      5331
😔      5244
😴      4999
      ...  
🕍         1
🕣         1
🈹         1
📳         1
🏺         1
📇         1
🈚         1
🦈         1
🤡         1
🏣         1
🎚         1
🔀         1
🏗         1
🦍         1
🥜         1
📭         1
🏬         1
㊙         1
🏤         1
🤠         1
🎽         1
📐         1
🔏         1
🗂         1
🚸         1
🎑         1
📯         1
📂         1
🔣         1
🏦         1
Name: emoji, Length: 989, dtype: int64

In [0]:
tweets = all_tweets.groupby('emoji').filter(lambda c:len(c) > 1000)
tweets['emoji'].value_counts()

😂    124823
❤     43218
😍     40566
😭     35714
😊     20076
🙄     17963
😩     16232
🔥     15453
🤔     15419
💕     12026
💯     11783
😘     11065
💀      9928
✨      9886
🙃      9405
👀      7842
😒      7019
☺      6871
😢      6846
😳      6716
💙      6616
😎      6349
😉      6272
😅      6133
😁      6010
😌      5759
😏      5623
💖      5331
😔      5244
😴      4999
      ...  
✌      1523
📸      1496
🎤      1487
🌚      1452
👅      1431
🏈      1373
🌟      1355
⏩      1332
❗      1325
🔴      1304
☕      1296
👊      1273
👇      1259
❣      1254
🎧      1246
🎈      1210
⏭      1198
💫      1181
↪      1157
🤑      1152
⚽      1141
😹      1112
😶      1108
💦      1075
😣      1074
😥      1072
🙁      1066
🤕      1065
☀      1013
😰      1013
Name: emoji, Length: 121, dtype: int64

In [0]:
max(tweets['text'], key=lambda t:len(t))

"Don't worry, my love, I don't get you wrong Don't get me wrong. We're connected with our hearts, souls, minds, bodies. If any doubt, we ask"

In [0]:
chars = list(sorted(set(chain(*tweets['text']))))
char_to_idx = {ch: idx for idx, ch in enumerate(chars)}
max_sequence_len = max(len(x) for x in tweets['text'])

emojis = list(sorted(set(tweets['emoji'])))
emoji_to_idx = {em: idx for idx, em in enumerate(emojis)}
emojis[:10]

train_tweets, test_tweets = train_test_split(tweets, test_size=0.1)


In [0]:
def data_generator(tweets, batch_size):
    while True:
        if batch_size is None:
            batch = tweets
            batch_size = batch.shape[0]
        else:
            batch = tweets.sample(batch_size)
        X = np.zeros((batch_size, max_sequence_len, len(chars)))
        y = np.zeros((batch_size,))
        for row_idx, (_, row) in enumerate(batch.iterrows()):
            y[row_idx] = emoji_to_idx[row['emoji']]
            for ch_idx, ch in enumerate(row['text']):
                X[row_idx, ch_idx, char_to_idx[ch]] = 1
        yield X, y

next(data_generator(tweets, 10))

(array([[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [1., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],
 
        ...,
 
        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],


## Featurizing and preparing our data

Just like we did when computing word embeddings, we want to featurize our data so we can classify it effectively.

In [0]:
VOCAB_SIZE = 100000
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(tweets['text'])

In [0]:
training_tokens = tokenizer.texts_to_sequences(train_tweets['text'])
test_tokens = tokenizer.texts_to_sequences(test_tweets['text'])
max_num_tokens = max(len(x) for x in chain(training_tokens, test_tokens))
training_tokens = pad_sequences(training_tokens, maxlen=max_num_tokens)
test_tokens = pad_sequences(test_tokens, maxlen=max_num_tokens)

In [0]:
training_labels = np.asarray([emoji_to_idx[em] for em in train_tweets['emoji']])
test_labels = np.asarray([emoji_to_idx[em] for em in test_tweets['emoji']])

In [0]:
import random
import twitter
import emoji
import gensim
import unicodedata
import html
from keras.preprocessing.text import text_to_word_sequence
import re



In [0]:
status_stream = twitter.TwitterStream(auth=auth).statuses
next(status_stream.sample()).keys()

dict_keys(['delete'])

In [0]:
RE_URL = re.compile(r'(http|ftp|https)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
RE_WHITESPACE = re.compile(r'\s+')

def strip_accents(s):
     return ''.join(c for c in unicodedata.normalize('NFD', s)
                    if unicodedata.category(c) != 'Mn')

class TokensYielder(object):
    def __init__(self, tweet_count, stream):
        self.tweet_count = tweet_count
        self.stream = stream

    def __iter__(self):
        print('!')
        count = self.tweet_count
        for tweet in self.stream:
            if tweet.get('lang') != 'en':
                continue
            text = tweet['text']
            text = html.unescape(text)
            text = RE_WHITESPACE.sub(' ', text)
            text = RE_URL.sub(' ', text)
            text = strip_accents(text)
            text = ''.join(ch for ch in text if ord(ch) < 128)
            if text.startswith('RT '):
                text = text[3:]
            text = text.strip()
            if text:
                yield text_to_word_sequence(text)
                count -= 1
                if count <= 0:
                    break

for t in TokensYielder(3, twitter.TwitterStream(auth=auth).statuses.sample()):
    print(t)

!
['amadayum', 'gogobebe9966', 'we', 'stan', 'talented', 'ladies', 'the', "album's", 'greatttt']
['enderski', 'redheads', 'are', 'nuts', 'badtz', 'i', 'like', 'my', 'girls', 'nuts']
['we', 'must', 'ensure', 'adequate', 'professional', 'and', 'technical', 'm', 'e', 'capacity', 'in', 'that', 'capacity', 'goes', 'down', 'to', 'the', 'district']


In [0]:
model = gensim.models.Word2Vec(tweets,workers=5,min_count=2)
model.save('twitter_stream_w2v.model')

In [0]:
def load_weights(tokenizer):
    model = Word2Vec.load('/content/gdrive/My Drive/glove.6B.100d.txt')
    w2v = np.zeros((tokenizer.num_words, w2v_model.syn0.shape[1]))
    for k, v in tokenizer.word_index.items():
        if v >= tokenizer.num_words:
            continue
        if k in w2v_model:
            w2v[v] = w2v_model[k]
    return w2v

In [0]:

#w2v = load_weights(tokenizer)

model = Word2Vec.load('twitter_stream_w2v.model')
w2v = np.zeros((tokenizer.num_words, model.wv.syn0.shape[1]))
found = 0
for k, v in tokenizer.word_index.items():
    if v >= tokenizer.num_words:
        continue
    if k in model:
        w2v[v] = model[k]
        found += 1
found, tokenizer.num_words


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


(2, 100000)

# World Level CNN


In [0]:
def create_cnn_model(vocab_size, embedding_size=None, embedding_weights=None, drop_out=0.2):
    message = Input(shape=(max_num_tokens,), dtype='int32', name='cnn_input')
    
    
    # The convolution layer in keras does not support masking, so we just allow
    # the embedding layer to learn an explicit value.
    embedding = Embedding(mask_zero=False, input_dim=vocab_size, 
                          output_dim=embedding_weights.shape[1], 
                          weights=[embedding_weights],
                          trainable=True,
                          name='cnn_embedding')(message)
    
    global_pools = []
    for window in 2, 3:
        conv_1x = Conv1D(128, window, activation='relu', padding='valid')(embedding)
        max_pool_1x = MaxPooling1D(2)(conv_1x)
        conv_2x = Conv1D(256, window, activation='relu', padding='valid')(max_pool_1x)
        max_pool_2x = MaxPooling1D(2)(conv_2x)
        conv_3x = Conv1D(256, window, activation='relu', padding='valid')(max_pool_2x)

        global_pools.append(GlobalMaxPooling1D()(conv_3x))

    merged = Concatenate(axis=1)(global_pools)
    fc1 = Dense(units=128, activation='relu')(merged)
    preds = Dense(units=len(emojis), activation='softmax', name='cnn_predictions')(fc1)
    model = Model(
        inputs=[message],
        outputs=[preds],
    )
    model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model

cnn_model = create_cnn_model(VOCAB_SIZE, embedding_weights=w2v)
cnn_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cnn_input (InputLayer)          (None, 54)           0                                            
__________________________________________________________________________________________________
cnn_embedding (Embedding)       (None, 54, 100)      10000000    cnn_input[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 53, 128)      25728       cnn_embedding[0][0]              
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 52, 128)      38528       cnn_embedding[0][0]              
_____________________________________

In [0]:
cnn_model.fit(training_tokens, training_labels, epochs=200,batch_size=10000)

Epoch 1/200
634695/634695 [==============================] - 63s 99us/step - loss: 3.5363 - acc: 0.2305
Epoch 2/200
634695/634695 [==============================] - 59s 93us/step - loss: 3.2956 - acc: 0.2645
Epoch 3/200
634695/634695 [==============================] - 59s 93us/step - loss: 3.0712 - acc: 0.3065
Epoch 4/200
634695/634695 [==============================] - 59s 93us/step - loss: 2.8554 - acc: 0.3509
Epoch 5/200
634695/634695 [==============================] - 59s 93us/step - loss: 2.6607 - acc: 0.3922
Epoch 6/200
634695/634695 [==============================] - 59s 93us/step - loss: 2.4904 - acc: 0.4284
Epoch 7/200
634695/634695 [==============================] - 59s 93us/step - loss: 2.3389 - acc: 0.4609
Epoch 8/200
634695/634695 [==============================] - 59s 93us/step - loss: 2.2057 - acc: 0.4898
Epoch 9/200
634695/634695 [==============================] - 59s 93us/step - loss: 2.0919 - acc: 0.5141
Epoch 10/200
634695/634695 [==============================] - 59

In [0]:
def create_lstm_model(vocab_size, embedding_size=None, embedding_weights=None):
    message = Input(shape=(None,), dtype='int32', name='lstm_input')
    embedding = Embedding(mask_zero=False, input_dim=vocab_size, 
                          output_dim=embedding_weights.shape[1], 
                          weights=[embedding_weights],
                          trainable=True,
                          name='lstm_embedding')(message)

    lstm_1 = LSTM(units=128, return_sequences=False)(embedding)
    preds = Dense(units=len(emojis), activation='softmax', name='lstm_predictions')(lstm_1)
    
    model = Model(
        inputs=[message],
        outputs=[preds],
    )
    model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model

In [0]:
lstm_model = create_lstm_model(VOCAB_SIZE, embedding_weights=w2v)
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_input (InputLayer)      (None, None)              0         
_________________________________________________________________
lstm_embedding (Embedding)   (None, None, 100)         10000000  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               117248    
_________________________________________________________________
lstm_predictions (Dense)     (None, 121)               15609     
Total params: 10,132,857
Trainable params: 10,132,857
Non-trainable params: 0
_________________________________________________________________


In [0]:
lstm_model.fit(training_tokens, training_labels, epochs=300, batch_size=10000)

Epoch 1/300
634695/634695 [==============================] - 49s 77us/step - loss: 3.9675 - acc: 0.1828
Epoch 2/300
634695/634695 [==============================] - 49s 77us/step - loss: 3.5340 - acc: 0.2425
Epoch 3/300
634695/634695 [==============================] - 48s 76us/step - loss: 3.2527 - acc: 0.2846
Epoch 4/300
634695/634695 [==============================] - 49s 77us/step - loss: 2.9923 - acc: 0.3320
Epoch 5/300
634695/634695 [==============================] - 49s 77us/step - loss: 2.7958 - acc: 0.3707
Epoch 6/300
634695/634695 [==============================] - 49s 77us/step - loss: 2.5991 - acc: 0.4102
Epoch 7/300
634695/634695 [==============================] - 49s 77us/step - loss: 2.4483 - acc: 0.4411
Epoch 8/300
634695/634695 [==============================] - 49s 77us/step - loss: 2.2987 - acc: 0.4722
Epoch 9/300
634695/634695 [==============================] - 49s 76us/step - loss: 2.1708 - acc: 0.4972
Epoch 10/300
634695/634695 [==============================] - 49

In [0]:
lstm_model.evaluate(test_tokens, test_labels)

70522/70522 [==============================] - 50s 709us/step


[7.0228537029296545, 0.37249368991321297]

## Comparing our models

Let's compare the predictions from our models on a sample of our data.

In [0]:
test_char_vectors, _ = next(data_generator(test_tweets, None)) 

In [0]:
predictions = {
    label: [emojis[np.argmax(x)] for x in pred]
    for label, pred in (
        ('lstm', lstm_model.predict(test_tokens[:100])),
        ('cnn', cnn_model.predict(test_tokens[:100])),
    )
}

In [0]:
# Make a dataframe just for test data
pd.options.display.max_colwidth = 128
test_df = test_tweets[:100].reset_index()
eval_df = pd.DataFrame({
    'content': test_df['text'],
    'true': test_df['emoji'],
    **predictions
})
eval_df[['content', 'true', 'cnn', 'lstm']].head(25)

,content,true,cnn,lstm
0,"@iAmydear: Done watching this episode. Mix emotions, but dominant is understanding and at peace. #ALDUBBlessedPair",❤,❤,❤
1,Congrats on SNL you and Tyler never fail to shooken me :),✨,✌,❤
2,@iiBreakNecks: dude playing dramatic music in the background while Cam'ron arguing with his girl. LEGENDARY,😂,😂,😂
3,All these denial letters I been getting,😩,😐,💀
4,@LeanOverHoes: You mad bitch well stay mad .,😂,😂,😂
5,@AvianaMonjae: A boogie x friend zone,😄,❤,❤
6,"@zenkmm: We fight,kiss,hug,text,argue,laugh,smile,love...that's us #ALDUBLoveEndures @DJGraphicsArts @fleurs7754 @xinempl",❤,☺,❤
7,"LunaLaCreme ""I officially would not be able to go to Victoria, Australia """,😆,😋,💀
8,@Maelle1505 @selijedhead @Nele_x3_ @lauraJEDsmiley Hope we can have a reunion soooooooon!,💖,💜,💕
9,@GagaMagazineUK: V Magazine have been loving Lady Gaga over the past few days,👀,😍,😚


## Qualitative Evaluation

We can examine some of our error cases by hand.  Often, the models tend to agree when they make mistakes, and that the mistakes aren't unreasonable: this task would be challenging even for a human.

In [0]:
eval_df[eval_df['lstm'] != eval_df['true']].head(10)

,cnn,content,lstm,true
1,✌,Congrats on SNL you and Tyler never fail to shooken me :),❤,✨
3,😐,All these denial letters I been getting,💀,😩
5,❤,@AvianaMonjae: A boogie x friend zone,❤,😄
7,😋,"LunaLaCreme ""I officially would not be able to go to Victoria, Australia """,💀,😆
8,💜,@Maelle1505 @selijedhead @Nele_x3_ @lauraJEDsmiley Hope we can have a reunion soooooooon!,💕,💖
9,😍,@GagaMagazineUK: V Magazine have been loving Lady Gaga over the past few days,😚,👀
11,😎,@Allykat728 oh yea and the people walking in and out of the woods in vests near by too,🙌,😭
13,😳,Hoh 80 deirr its peanuts,😭,😂
14,➡,Let's flirt on webcam O <== Just find me there,👉,⏭
18,😂,I DID!!!,😭,😊


In [0]:
def combined_data_generator(tweets, tokens, batch_size):
    tweets = tweets.reset_index()
    while True:
        batch_idx = random.sample(range(len(tweets)), batch_size)
        tweet_batch = tweets.iloc[batch_idx]
        token_batch = tokens[batch_idx]
        char_vec = np.zeros((batch_size, max_sequence_len, len(chars)))
        token_vec = np.zeros((batch_size, max_num_tokens))
        y = np.zeros((batch_size,))
        for row_idx, (token_row, (_, tweet_row)) in enumerate(zip(token_batch, tweet_batch.iterrows())):
            y[row_idx] = emoji_to_idx[tweet_row['emoji']]
            for ch_idx, ch in enumerate(tweet_row['text']):
                char_vec[row_idx, ch_idx, char_to_idx[ch]] = 1
            token_vec[row_idx, :] = token_row
        yield { 'cnn_input': token_vec, 'lstm_input': token_vec}, y

d, y = next(combined_data_generator(train_tweets, training_tokens, 5))
d['lstm_input'].shape

(5, 54)

In [0]:
def prediction_layer(model):
    layers = [layer for layer in model.layers if layer.name.endswith('_predictions')]
    return layers[0].output

def create_ensemble(*models):
    inputs = [model.input for model in models]
    predictions = [prediction_layer(model) for model in models]
    merged = Average()(predictions)
    model = Model(
        inputs=inputs,
        outputs=[merged],
    )
    model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model


ensemble = create_ensemble(cnn_model, lstm_model)
ensemble.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
cnn_input (InputLayer)          (None, 54)           0                                            
__________________________________________________________________________________________________
cnn_embedding (Embedding)       (None, 54, 100)      10000000    cnn_input[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 53, 128)      25728       cnn_embedding[0][0]              
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 52, 128)      38528       cnn_embedding[0][0]              
__________________________________________________________________________________________________
max_poolin

In [0]:
BATCH_SIZE = 10000
ensemble.fit_generator(
    combined_data_generator(train_tweets, training_tokens, BATCH_SIZE),
    epochs=100,
    steps_per_epoch=len(train_tweets) / BATCH_SIZE,
    verbose=2
)

Epoch 1/100


In [0]:
ensemble.evaluate_generator(
    combined_data_generator(test_tweets, test_tokens, BATCH_SIZE),
    steps=len(test_tweets) / BATCH_SIZE
)

In [0]:
len(train_tweets)

634695

In [0]:
data = [["I love you"],["superb"],["hey"],["hello"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],       ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ["Thank you for dinner!"],      ["Thank you for dinner!"],["I don't like it"],["My car skidded on the wet street"],
       ]
df = pd.DataFrame(data,columns=['text'])
(df !='n').astype(int)
df


,text
0,I love you
1,superb
2,hey
3,hello
4,I don't like it
5,My car skidded on the wet street
6,Thank you for dinner!
7,I don't like it
8,My car skidded on the wet street
9,Thank you for dinner!


In [0]:
import numpy as np
df_tokens=tokenizer.texts_to_sequences(df['text'])
df_tokens = pad_sequences(df_tokens, maxlen=max_num_tokens)
predictions = {
label: [emojis[np.argmax(x)] for x in pred]
for label, pred in (
('cnn_model', cnn_model.predict(df_tokens[:100])),
)
}

In [0]:
pd.options.display.max_colwidth = 128
text = df.reset_index()
eval_df = pd.DataFrame({
    'content': df['text'],
    **predictions
})
eval_df[['content',  'cnn_model']]

,content,cnn_model
0,I love you,❤
1,superb,😀
2,hey,😍
3,hello,😊
4,I don't like it,😂
5,My car skidded on the wet street,😂
6,Thank you for dinner!,😘
7,I don't like it,😂
8,My car skidded on the wet street,😂
9,Thank you for dinner!,😘
